In [3]:
import pandas as pd
import numpy as np
import os
from collections import Counter

In [218]:
column_names = ['0.0_1.0', '0.1_0.9', '0.2_0.8', '0.3_0.7', '0.4_0.6']
column_names.append("total_columns")

In [232]:
r_e_dict = {}
positive_negative = {}
for item in os.listdir():
    if item.startswith('abstract'):
        abs_df = pd.read_excel(item)
        r_e_list = []
        for i in abs_df.columns:
            if i.startswith('r.e'):
                r_e_list.append(abs_df[f'{i}'].sum())
            if i.startswith('Color'):
                positive = sum(abs_df['Color'] > 0)
                negative = len(abs_df) - positive
        r_e_list.append(sum(r_e_list)) 
        key_name = item.split(".")[0] + '_' + abs_df['Date'].iloc[0]
        r_e_dict[key_name] = r_e_list
        positive_negative[abs_df['Date'].iloc[0]] = [round(positive/(len(abs_df)), 3), round(negative/(len(abs_df)), 3)]

In [241]:
positive_negative

{'2016-06-03': [0.421, 0.579],
 '2016-06-06': [0.412, 0.588],
 '2016-06-07': [0.391, 0.609],
 '2016-06-08': [0.467, 0.533],
 '2016-06-09': [0.217, 0.783],
 '2016-06-10': [0.217, 0.783]}

In [234]:
pivot_dict = {}
after_pivot = {}
for item in os.listdir():
    if item.startswith('pivot_'):
        piv_df = pd.read_excel(item)
        pivot_list = []
        for i in range(len(piv_df)-1):
            if piv_df['total'].iloc[i] == piv_df['total'].iloc[i+1]:
                pivot_list.append(piv_df['total'].iloc[i])
        
        counts = Counter(pivot_list)
        for number, count in counts.items():
            if count > 1:
                pivot_list.append(number)
        pivot_list.sort(reverse=True)
        max_pivot = max(pivot_list)
        counts = Counter(pivot_list)

        result = []
        temp_after_pivot = []
        for number, count in counts.items():
            if count > 1:
                result.append(f"{number}_" + "_".join([str(number)] * (count - 1)))
            else:
                result.append(str(number)) 
        temp_after_pivot.append(piv_df[piv_df['minimum_area'] == piv_df['minimum_area'].min()]['time'].values[0])
        temp_after_pivot.append(round(piv_df['minimum_area'].min(), 4))
        temp_after_pivot.append(max_pivot)
        temp_after_pivot.append(sum(pivot_list))
        after_pivot[item] = temp_after_pivot
        pivot_dict[item] = result

In [240]:
piv_df['date'].iloc[0]

'2016-06-10'

In [154]:
max_len_piv_list = []
for item in pivot_dict.values():
    max_len_piv_list.append(len(item))

maximum_value = max(max_len_piv_list)

In [220]:
for i in range(maximum_value):
    column_names.append(f"pivot_{i+1}")

In [222]:
column_names.append("minimu def area")

In [225]:
column_names.insert(0, "date")

In [227]:
column_names.insert(0, "row_id")

In [155]:
final_pivot_dict = {}
for i, j in pivot_dict.items():
    pippo = j
    for _ in range(maximum_value - len(j)):
        pippo.extend(".")
    final_pivot_dict[i] = pippo

In [156]:
final_pivot_dict

{'pivot_1.xlsx': ['24', '.', '.', '.', '.', '.'],
 'pivot_2.xlsx': ['12', '10_10_10', '7_7_7', '.', '.', '.'],
 'pivot_3.xlsx': ['29', '27', '18', '6', '1', '0'],
 'pivot_4.xlsx': ['35', '32', '1', '0', '.', '.'],
 'pivot_5.xlsx': ['12', '7_7_7', '5', '4', '1', '.'],
 'pivot_6.xlsx': ['22', '21_21_21_21_21', '0_0_0', '.', '.', '.']}

In [188]:
after_pivot

{'pivot_1.xlsx': ['08:30:00', 0.0318, 24, 24],
 'pivot_2.xlsx': ['04:30:00', 0.0172, 12, 63],
 'pivot_3.xlsx': ['13:30:00', 0.0235, 29, 81],
 'pivot_4.xlsx': ['10:30:00', 0.0699, 35, 68],
 'pivot_5.xlsx': ['14:30:00', 0.0542, 12, 43],
 'pivot_6.xlsx': ['23:30:00', 0.0576, 22, 127]}

In [207]:
final_list = {}
for i in range(len(r_e_dict)):
    key_name = (list(positive_negative.keys())[i])
    pippo_list = [key_name]
    pippo_list.extend((list(r_e_dict.values())[i]))
    pippo_list.extend((list(final_pivot_dict.values())[i]))
    pippo_list.extend((list(after_pivot.values())[i]))
    pippo_list.extend((list(positive_negative.values())[i]))
    
    final_list[f"row_{i+1}"] = pippo_list

In [230]:
df = pd.DataFrame.from_dict(final_list, orient='index')

# Optionally, reset the index and assign proper column names
df.reset_index(inplace=True)
# df.columns = ['row_id', 'date', 'val1', 'val2', 'val3', 'val4', 'val5', 'val6', 'field_1', 'field_2', 'field_3', 
#               'field_4', 'field_5', 'field_6', 'time', 'metric_1', 'metric_2', 'metric_3', 'metric_4', 'metric_5']


df.columns = column_names

df = df[column_names[1:]]
print(df)

         date  0.0_1.0  0.1_0.9  0.2_0.8  0.3_0.7  0.4_0.6  total_columns  \
0  2016-06-03     -6.0    -29.0    -18.0    -14.0      5.0          -62.0   
1  2016-06-06     -3.0     -2.0     -2.0      0.0      0.0           -7.0   
2  2016-06-07     17.0     19.0     25.0     26.0      9.0           96.0   
3  2016-06-08     14.0      8.0     14.0     -5.0      1.0           32.0   
4  2016-06-09    -13.0    -20.0    -13.0     -4.0      0.0          -50.0   
5  2016-06-10    -44.0    -20.0      7.0      0.0     -1.0          -58.0   

  pivot_1         pivot_2 pivot_3 pivot_4 pivot_5 pivot_6 minidef pivote  \
0      24               .       .       .       .       .       08:30:00   
1      12        10_10_10   7_7_7       .       .       .       04:30:00   
2      29              27      18       6       1       0       13:30:00   
3      35              32       1       0       .       .       10:30:00   
4      12           7_7_7       5       4       1       .       14:30:00   
5   